# TASK 3 - STREAMING CIPHER

### 1. Define an object that, starting from a random bit generator, generates random bytes. 

In [72]:
from lfsr import LFSR
from itertools import islice
import random

class PseudoRandomByteGenerator(object):
    ''' class docstring '''

    def __init__(self, seed=None, bit_generator=None, **kwargs):

        self.poly = kwargs['poly']
        self.seed = seed

        """
        if self.seed == None: 
            self.seed = '0' * max(self.poly)
        while len(self.seed) < max(self.poly): 
            self.seed = '0' + self.seed
        """
        print(self.seed)

        self.bit_generator = bit_generator
        if self.bit_generator == None:
            self.bits = []
            self.lfsr_inst = LFSR(poly=kwargs['poly'], state=self.seed)#her er det noe feil. med 111 som seed blir lfsr_inst.s false,false,false...?
            print(self.lfsr_inst.s)
            print(self.lfsr_inst.p)
        elif self.bit_generator == 'random':
            random.seed(self.seed)
            self.bits = 0

    def __iter__(self):
        return self

    def bit_to_bytes(self, s):
        return int(s, 2).to_bytes(len(s) // 8, byteorder='big')
    
    def __next__(self):
        if self.bit_generator == None: 
            self.bits = self.lfsr_inst.run_steps(N=8)   #get a list of 8 bits represented as true and false
            self.bits = ''.join(['1' if b==True else '0' for b in self.bits])   #convert the list of bits to a string of 0 and 1
            byte = int(self.bits, 2)
        elif self.bit_generator == 'random':
            self.bits = random.getrandbits(8)
            byte = self.bits
        return byte

In [73]:
prbg = PseudoRandomByteGenerator(seed='111', poly=[3, 1, 0]) # iterator
digits = [byte for byte in islice(prbg, 10)]
print(bytes(digits))

111
[False, False, False]
[True, False, True]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [26]:
prbg = PseudoRandomByteGenerator(bit_generator='random') # iterator
digits = [byte for byte in islice(prbg, 10)]
print(bytes(digits)) # -> b'\x00x\xe7/\xdd7\xe1\\t\xc5'

b'!u\x13Tm\xe28p\xef\xbd'


### 2. Define an object implementing a Stream Cipher that, given a Pseudo Random Byte Generator and a key, can encrypt and decrypt a message. 

In [27]:
class StreamCipher(PseudoRandomByteGenerator):
    ''' class docstring '''
    def __init__(self, key, prng=None, **kwargs):
        
        super().__init__(seed=key, bit_generator=prng, **kwargs)
    
    def encrypt(self, plaintext):
        N = len(plaintext)
        digits = [byte for byte in islice(self.__iter__(), N)]
        encrypted = [a ^ b for (a,b) in zip(bytes(digits), plaintext)]
        ciphertext = bytes(encrypted)
        return ciphertext
    
    def decrypt(self, ciphertext):
        N = len(ciphertext)
        digits = [byte for byte in islice(self.__iter__(), N)]
        decrypted = [a ^ b for (a,b) in zip(bytes(digits), ciphertext)]
        plaintext = bytes(encrypted)
        return plaintext

In [ ]:
message = 'hello world!'
key = 0x0123456789ABCDEF
alice = StreamCipher(key, poly=[7, 1, 0])
plaintextA = message.encode('utf-8')
ciphertext = alice.encrypt(plaintextA)
bob = StreamCipher(key, poly[7, 1, 0])
plaintextB = bob.encrypt(ciphertext)

print(plaintextA) # -> b'hello world!'
print(ciphertext) # -> b'\x9f\x03\xbcf\xfa\xdb`\xf6\x17\xce7\x88'
print(plaintextB) # -> b'hello world!'

### 3. Define an iterator that implements the A5/1 architecture and use it as bit generator in a Stream Cipher

### 4. Define an iterator that implements the RC4 architecture and use it in the Stream Cipher object to encrypt/decrypt a message